# Deal with Multi-omics/Multi-view scCCC

In [1]:
import numpy as np
import pandas as pd
import scanpy as sc
import muon as mu
import liana as li

/home/dbdimitrov/anaconda3/envs/spiana/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from itertools import product

In [3]:
from liana.testing._sample_anndata import generate_toy_mdata
mdata = generate_toy_mdata()
mdata.mod['adata_y'].var.index = 'scaled:' + mdata.mod['adata_y'].var.index
sc.pp.scale(mdata.mod['adata_y'])

/home/dbdimitrov/anaconda3/envs/spiana/lib/python3.10/site-packages/mudata/_core/mudata.py:458: UserWarning: Cannot join columns with the same name because var_names are intersecting.


In [4]:
interactions = list(product(mdata.mod['adata_x'].var.index, mdata.mod['adata_y'].var.index))

In [5]:
resource = pd.DataFrame(interactions, columns=['ligand', 'receptor'])

In [6]:
resource

,ligand,receptor
0,HES4,scaled:HES4
1,HES4,scaled:TNFRSF4
2,HES4,scaled:SSU72
3,HES4,scaled:PARK7
4,HES4,scaled:RBP7
...,...,...
95,EFHD2,scaled:SRM
96,EFHD2,scaled:MAD2L2
97,EFHD2,scaled:AGTRAP
98,EFHD2,scaled:TNFRSF1B


In [10]:
li.mt.rank_aggregate(mdata, groupby='bulk_labels', mod_x='adata_x', mod_y='adata_y', use_raw=False, interactions=interactions, verbose=True, transform=None)

Transforming data to zero-inflated min-max scale
Using `.X`!
20 cells are empty, they will be removed.
Using provided `interactions`.
Generating ligand-receptor stats for 680 samples and 20 features
Assuming that counts were `natural` log-normalized!


/home/dbdimitrov/anaconda3/envs/spiana/lib/python3.10/site-packages/pandas/core/indexing.py:1728: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.


Running CellPhoneDB


100%|██████████| 1000/1000 [00:03<00:00, 276.92it/s]


Running Connectome
Running log2FC
Running NATMI
Running SingleCellSignalR
Running CellChat


100%|██████████| 1000/1000 [00:02<00:00, 362.89it/s]


In [8]:
# TODO check how much do aggregate ranks agree
mdata.uns['liana_res']

,source,target,ligand_complex,receptor_complex,lr_means,cellphone_pvals,expr_prod,scaled_weight,lr_logfc,spec_weight,lrscore,lr_probs,cellchat_pvals,specificity_rank,magnitude_rank
1483,CD4+/CD45RO+ Memory,CD4+/CD45RO+ Memory,PARK7,scaled:PARK7,0.563684,0.015,0.317740,0.461206,0.165597,0.015466,0.783310,0.426254,0.005,0.004185,2.320225e-10
1552,CD8+ Cytotoxic T,CD4+/CD45RO+ Memory,PARK7,scaled:PARK7,0.534070,0.001,0.284354,0.343557,0.129845,0.013841,0.773740,0.411399,0.000,0.001630,1.855840e-09
2143,CD4+/CD45RO+ Memory,CD8+ Cytotoxic T,PARK7,scaled:PARK7,0.534070,0.001,0.284354,0.343557,0.129845,0.013841,0.773740,0.411399,0.000,0.001630,6.262315e-09
1926,Dendritic,CD4+/CD45RO+ Memory,PARK7,scaled:PARK7,0.515404,0.003,0.263311,0.269403,0.101287,0.012817,0.766940,0.386869,0.000,0.005797,2.898159e-08
2212,CD8+ Cytotoxic T,CD8+ Cytotoxic T,PARK7,scaled:PARK7,0.504456,0.041,0.254476,0.225908,0.094094,0.012387,0.763876,0.396704,0.002,0.016871,3.839431e-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4653,CD4+/CD25 T Reg,CD34+,SSU72,scaled:RBP7,0.000000,1.000,0.000000,-0.068141,-0.012297,0.000000,0.000000,0.000000,1.000,1.000000,1.000000e+00
4659,CD4+/CD25 T Reg,CD34+,EFHD2,scaled:RBP7,0.000000,1.000,0.000000,-0.254700,-0.107129,0.000000,0.000000,0.000000,1.000,1.000000,1.000000e+00
4654,CD4+/CD25 T Reg,CD34+,PARK7,scaled:RBP7,0.000000,1.000,0.000000,-0.118316,-0.033078,0.000000,0.000000,0.000000,1.000,1.000000,1.000000e+00
4656,CD4+/CD25 T Reg,CD34+,MAD2L2,scaled:RBP7,0.000000,1.000,0.000000,-0.167202,-0.056575,0.000000,0.000000,0.000000,1.000,1.000000,1.000000e+00


In [ ]:
from liana.funcomics.mdata_to_anndata import mdata_to_anndata

In [ ]:
adata = mdata_to_anndata(mdata, mod_x='adata_x', mod_y='adata_y', transform=False)

In [ ]:
adata.is_view

In [ ]:
li.mt.rank_aggregate(adata, groupby='bulk_labels', use_raw=False, resource=resource, verbose=True)

## li.mt.multiview(MuData, x_mod, y_mod, interactions, transform_fun=None, ...)

In [ ]:
from anndata import concat

In [ ]:
from scipy.sparse import csr_matrix, isspmatrix_csr

In [ ]:
def zi_minmax(X, cutoff=0.1):
    X = X.copy()
    # Ensure the matrix is in Compressed Sparse Row (CSR) format
    if not isspmatrix_csr(X):
        X = csr_matrix(X)

    # Min-Max scaling on non-zero elements
    min_vals = np.array(X.min(axis=0).todense())[0]
    max_vals = np.array(X.max(axis=0).todense())[0]
    nonzero_rows, nonzero_cols = X.nonzero()
    scaled_values = (X.data - min_vals[nonzero_cols]) / (max_vals[nonzero_cols] - min_vals[nonzero_cols])
    
    # Apply cutoff to the matrix
    scaled_values[scaled_values < cutoff] = 0
    nonzero_rows, nonzero_cols = X.nonzero()

    # Create a new sparse matrix with scaled values
    X = csr_matrix((scaled_values, (nonzero_rows, nonzero_cols)), shape=X.shape)

    return X

In [ ]:
## Refactor slightly liana pipe to accept interactions (i.e. list of tuples?)

In [ ]:
# Then this just becomes a wrapper around the liana pipe, where the method of interest is passed as an argument? + We do zi_minmax to deal with different scales
# Then change select_resource to return tuples instead of a dataframe, and dataframe when df=True

In [ ]:
# We take the two modalities and stack as values to the same AnnData
# transform_fun will be None and it can be applied by columns (i.e. by variable); e.g. lambda x: zi_minmax(x, axis=0)

In [ ]:
interactions = list(product(mdata.mod['adata_x'].var.index, mdata.mod['adata_y'].var.index))

In [ ]:
resource = pd.DataFrame(interactions, columns=['ligand', 'receptor'])

Actual function

In [ ]:
def _mdata_to_anndata(mdata, mod_x, mod_y, transform=True):
    if mod_x == mod_y:
        raise ValueError('The two modalities must be different')
    if mod_x not in mdata.mod.keys():
        raise ValueError('The first modality is not in the mdata')
    if mod_y not in mdata.mod.keys():
        raise ValueError('The second modality is not in the mdata')
    
    md_x = mdata.mod[mod_x]
    md_y = mdata.mod[mod_y]
    
    adata = concat([md_x, md_y], join='outer', axis=1, merge='first', label='modality')
    
    if transform:
        adata.X = zi_minmax(adata.X)
    
    return adata
    

In [ ]:
if isinstance(mdata, mu.MuData):
    adata = _mdata_to_anndata(mdata, 'adata_x', 'adata_y', transform=True)

In [ ]:
adata

In [ ]:
mod_x = 'adata_x'
mod_y = 'adata_y'

In [ ]:
mdata.mod[mod_x].X = mdata.mod[mod_x].X.astype(np.float32)
mdata.mod[mod_y].X = mdata.mod[mod_x].X.astype(np.float32)

In [ ]:
# Concat the two modalities
adata = concat([mdata.mod['adata_x'], mdata.mod['adata_y']], join='outer', axis=1, merge='first')

Apply minmax

In [ ]:
# adata.X = zi_minmax(adata.X)
# sc.pp.filter_cells(adata, min_genes=1)

In [ ]:
li.mt.cellphonedb(adata, resource=resource, groupby='bulk_labels', use_raw=False, verbose=True)

In [ ]:
adata = sc.datasets.pbmc68k_reduced()

In [ ]:
li.mt.rank_aggregate(adata, groupby='bulk_labels', expr_prop=0.1, verbose=True, use_raw=True)

In [ ]:
liana_res = adata.uns['liana_res'].copy()

In [ ]:
adata = adata.raw.to_adata()

In [ ]:
adata.X = zi_minmax(adata.X)

In [ ]:
li.mt.cellphonedb(adata, groupby='bulk_labels', expr_prop=0.1, verbose=True, use_raw=False)

In [ ]:
minmax_res = adata.uns['liana_res']

In [ ]:
liana_res

In [ ]:
minmax_res['cellphone_pvals']

In [ ]:
minmax_res.rename(columns={'cellphone_pvals': 'cellphone_pvals_minmax', 
                           'lr_means':'lr_minmax'}, inplace=True)

In [ ]:
merged = minmax_res.merge(liana_res, how='inner', on=['ligand_complex', 'receptor_complex', 'source', 'target'])

In [ ]:
pearsonr(merged['cellphone_pvals'], merged['cellphone_pvals_minmax'])

In [ ]:
pearsonr(merged['lr_means'], merged['lr_minmax'])

## li.fun.estimate_metalinks(adata, est_fun ...) 
returns (metabolite_estimates:csr_matrix, receptors_masked:csr_matrix)

## li.mt.metalinks(adata, groupby, resource, ...)